In [ ]:
%matplotlib inline
import sys
sys.path.append("..") # Adds the module to path

# deeptrack.properties

This notebook introduces the module deeptrack.properties.

## What is a property?

Each feature (instance of the class `Feature`, see [features_example](features_example.ipynb)) can have several properties (instances of the class `Property`).
A propety has a value accessible through the field `current_value`, whose data type is not restricted. 
This value is updated through a sampling rule (method `.update()`), which is passed to the class constructor on initialization. 

## What is a sampling rule?

The sampling rule determines how the value of a property is updated upon calling `.update()`.
A sampling rule is defined when an instance of the class Property is created and can be of any type. 
When calling `.update()`, the value of the property is updated according to the first of the following that applies:
    
1.  If the sampling rule has a method `.sample()`, call `.sample()` and return the output.
2.  If the sampling rule is a dictionary, create an exact copy of the dictionary and substitute
    any value that has a `.sample()` method with the result of the call to this method.
3.  If the sampling rule is either a `list` or a 1-dimensional `ndarray`, extract one element randomly.
4.  If the sampling rule is an iterable, return the next value.
5.  If the sampling rule is callable, call it with no arguments and return the result.
6.  If none of the above apply, return the sampling rule itself.

In [ ]:
import numpy as np
from deeptrack.properties import Property

## 1 - Property with a constant value

The simplest example of a property is one that does not change during an update call.
This is commonly either a number or a tuple, but can be any data type that will be evaluated by case 6.
If you want to have a constant property with a value that would be evaluated by cases 1-5 (e.g., a list or a function), you can  wrap it as the output of a lambda function.

In [ ]:
# NUMBER
print("NUMBER")

P = Property(1)
print("The current value of the property is", P.current_value)

P.update() # Numbers are not changed after a update() call
print("The current value of the property is", P.current_value)

In [ ]:
# TUPLE
print("TUPLE")

P = Property((1, [2, 3], None))
print("The current value of the property is", P.current_value)

P.update() # Tuples are not changed after a update() call
print("The current value of the property is", P.current_value)

In [ ]:
# WRAPPED LIST
print("WRAPPED LIST")

P = Property(lambda: [1, 2, 3])
print("The current value of the property is", P.current_value)

P.update() # Tuples are not changed after a update() call
print("The current value of the property is", P.current_value)

## 2 - Property with a discrete random value 

Discrete randomness can be achieved by a list (case 3), a 1-dimensional ndarray (case 3), or a function (case 5). For lists and ndarrays, the output is a single element of the list chosen uniform-randomly. For non-uniform sampling, either use lists with repeated elements, or a function. 

In [ ]:
# LIST
print("LIST")

P = Property([1, 2, 3, 4])
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

In [ ]:
# NDARRAY
print("NDARRAY")

P = Property(np.linspace(0, 1, 5))
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)

In [ ]:
# FUNCTION
print("FUNCTION")

P = Property(lambda: 1 if np.random.rand() > 0.6 else 0)
for _ in range(5): 
    P.update()
    print("The current value of the property is", P.current_value)

## 3 - Property with a continuous random value 

Continuous randomness is typically achieved by passing a function that returns a coninuous random value. This function should take no input, as noted in case 5. To use a function that needs arguments, wrap it in a function that calls it with the correct arguments.

In [ ]:
# FUNCTION WITH NO INPUT
print("FUNCTION WITH NO INPUT")

P = Property(np.random.rand)
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

In [ ]:
# WRAPPED FUNCTION
print("WRAPPED FUNCTION")

P = Property(lambda: np.random.normal(1, 5))
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

## 4 - Property with a deterministically changing value

Deterministically changing properties can be achieved using either an iterator (case 4) or a function (case 5). For the output of a function to change deterministically between calls, it should reference some variable outside its definition. Once an iterator has been exhausted, it will repeat the last value.

In [ ]:
# ITERATOR
print("ITERATOR")

P = Property(iter([1, 2, 3, 4, 5]))
for _ in range(10):
    P.update()
    print("The current value of the property is", P.current_value)

In [ ]:
# FUNCTION
print("FUNCTION")

fibbonacci = [1, 1]
def fibbonacci_sequence():
    fibbonacci.append(fibbonacci[-2] + fibbonacci[-1])
    return fibbonacci

P = Property(fibbonacci_sequence)
for _ in range(5):
    P.update()
    print("The current value of the property is", P.current_value)

## 6. What is a PropertyDict?

The second class contained in the deeptrack.properties module is called PropertyDict. This is a dictionary of properties (keys: name of properties; values: properties) complemented by utility methods to manage collections of properties. These include:

* `.current_value_dict()`, which creates and returns a dictionary with the current value of all properties in the PropertyDict (keys: name of properties; values: current values of the properties).
* `.update()`, which calls the method `.update()` on all properties in the PropertyDict.
* `.sample()`, which calls the method `.sample()` on all properties in the PropertyDict, and creates and returns a dictionary from the output (keys: name of properties; values: sample outputs of the properties).


In [ ]:
from deeptrack.properties import PropertyDict, Property

property_dict = PropertyDict(
    foo=Property(1),
    bar=Property([1, 2, 3]),
    baz=Property(np.random.rand)
)

for _ in range(5):
    property_dict.update()
    print("The current values of the properties in property_dict are", property_dict.current_value_dict())